In [30]:
import random
import re
import pandas as pd
import numpy as np
import joblib

class Forecast:

    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
#        self.list_of_ingredients = self.list_of_ingredients.lower()
#        self.list_of_ingredients = self.list_of_ingredients.split(',')
#        self.list_of_ingredients = [x.strip(' ') for x in self.list_of_ingredients]

    def preprocess(self):
        df = pd.read_csv('epi_r_clean.csv')
        groceries = df.columns[2:]
        vector = pd.DataFrame(data = np.zeros((1, len(groceries))), columns=groceries)
        for i in list(groceries):
            if i in self.list_of_ingredients:
                vector[i] = 1.0
        self.vector = vector
        return self.vector

    def predict_rating_category(self):
        model = joblib.load('best_RandomForestClassifier_cl.joblib')
        rating_cat = model.predict(self.vector)
        if rating_cat == 'great':
            text = 'Вкусное. Отличный выбор продуктов!'
        if rating_cat == 'so-so':
            text = 'Нормальное. Но может пересмотрим что-то в списке продуктов?'
        if rating_cat == 'bad':
            text = 'Невкусное. Хоть конкретно Вам может быть и понравится блюдо из этих ингридиентов, но, на наш взгляд, это плохая идея - готовить блюдо из них. Хотели предупредить.'
        rating_cat = rating_cat
        return rating_cat, text

class NutritionFacts:
    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
#        self.list_of_ingredients = self.list_of_ingredients.lower()
#        self.list_of_ingredients = self.list_of_ingredients.split(',')
#        self.list_of_ingredients = [x.strip(' ') for x in self.list_of_ingredients]
        
    def retrieve(self):
        df = pd.read_csv('df_nutrition.csv')
        for i in self.list_of_ingredients:
            if i == self.list_of_ingredients[0]:
                nutr = df.loc[df['food_product'] == i]
                df_nutr = nutr
            else:
                nutr = df.loc[df['food_product'] == i]
                df_nutr = pd.concat([df_nutr, nutr])
        df_nutr.reset_index(drop=True, inplace=True)
        for x in range(len(df_nutr)):
            df_nutr['pr_daily_norm'][x] =  round(df_nutr['pr_daily_norm'][x], 0)
            df_nutr['nutrition'][x] = df_nutr['nutrition'][x].split(',')[0]
        df_nutr = df_nutr.loc[df_nutr['pr_daily_norm'] > 0]
        df_nutr = df_nutr.sort_values(by=['food_product', 'pr_daily_norm'], ascending=False)
        df_nutr.drop(columns=['value'], inplace=True)
        df_nutr.reset_index(drop=True, inplace=True)
        self.facts = df_nutr
        return self.facts

class SimilarRecipes:
    def __init__(self, list_of_ingredients):
        self.list_of_ingredients = list_of_ingredients
#        self.list_of_ingredients = self.list_of_ingredients.lower()
#        self.list_of_ingredients = self.list_of_ingredients.split(',')
#        self.list_of_ingredients = [x.strip(' ') for x in self.list_of_ingredients]
    
    def find_all(self):
        df = pd.read_csv('df_recipes.csv')
        df = df.loc[df['val'] - len(self.list_of_ingredients) <= 5]
        for i in self.list_of_ingredients:
            if i == self.list_of_ingredients[0]:
                df_rec = df.loc[df[i] == 1]
                df_recipes = df_rec
            else:
                df_rec = df.loc[df[i] == i]
                df_recipes = pd.concat([df_recipes, df_rec])
        df_recipes = df_recipes.drop_duplicates(keep='first')
        df_recipes.reset_index(drop=True, inplace=True)
        df_recipes['ing'] = df_recipes['val']
        df_recipes['ing_n'] = df_recipes['val']
        for x in range(len(df_recipes)):
            df_recipes['ing'][x] = 0
            for y in self.list_of_ingredients:
                df_recipes['ing'][x] = df_recipes['ing'][x] + df_recipes[y][x]
        df_recipes['ing_n'][x] = df_recipes['val'][x] - df_recipes['ing'][x]
        df_recipes = df_recipes.sort_values(by = ['ing', 'ing_n'], ascending=[False, True])
        self.n = df_recipes['ing'][0]
        self.indexes = df_recipes
        return self.indexes, self.n

    def top_similar(self):
        text_with_recipes = self.indexes
        text_with_recipes = text_with_recipes.loc[text_with_recipes['ing'] <= 5]
        text_with_recipes.reset_index(drop=True, inplace=True)
        return text_with_recipes

In [33]:
df_1 = pd.read_csv('epi_r_clean.csv')
groceries = df.columns[2:]; groceries

Index(['alcoholic', 'almond', 'amaretto', 'anchovy', 'anise', 'apple',
       'apple juice', 'apricot', 'artichoke', 'asian pear',
       ...
       'whiskey', 'white wine', 'whole wheat', 'wild rice', 'wine', 'wok',
       'yellow squash', 'yogurt', 'zucchini', 'turkey'],
      dtype='object', length=389)

saffron, gin, chile pepper - bad // 
bread, avocado, cheese, garlic - great //
a, b, c, garlic - a,b,c - нет в базе//
bread, avocado, cheese, garlic, saffron, gin, chile pepper, banana, yogurt, zucchini - рецептов не найдено

In [24]:
list_of_ingredients = input("Please write name of the ingredient in English, do not use numbers or special characters: ")
list_of_ingr = list_of_ingredients.lower()
list_of_ingr = list_of_ingr.split(',')
list_of_ingr = [x.strip(' ') for x in list_of_ingr]

In [25]:
list_of_ingr

['saffron', 'gin', 'chile pepper']

In [32]:
import pandas as pd
pd.options.mode.chained_assignment = None
import re
print("Ingredients List Example: apple, bean, cookie, fruit, lentil, tomato")
list_of_ingredients = input("Please write name of the ingredient in English, do not use numbers or special characters: ")
list_of_ingr = list_of_ingredients.lower()
list_of_ingr = list_of_ingr.split(',')
list_of_ingr = [x.strip(' ') for x in list_of_ingr]

fore = Forecast(list_of_ingr)
nutr = NutritionFacts(list_of_ingr)
simi = SimilarRecipes(list_of_ingr)

df = pd.read_csv('epi_r_clean.csv')
groceries = df.columns[2:]
if len(list_of_ingr) <= 1:
    print("Слишком мало продуктов. Введите 2 продукта и более")
elif len(list_of_ingr) > 1:
    a = 0
    for i in list_of_ingr:
        if i not in list(groceries):
            print("К сожелению, не знаю такой продукт:", i)
        else: a = a + 1
    if a < len(list_of_ingr):
        print("Попробуйте выбрать что-нибудь из списка:", list(groceries))
    if a == len(list_of_ingr):
        recipe = fore.recipes(list_of_ingr)
        recipe.preprocess()
        rating_cat, text = recipe.predict_rating_category()
        print('\n' + 'I. НАШ ПРОГНОЗ' + '\n' + text + '\n')
        nutrition = recipes.NutritionFacts(list_of_ingr)
        facts = nutrition.retrieve()
        print("II. ПИЩЕВАЯ ЦЕННОСТЬ")
        for n in list_of_ingr:
            print('\n' + n + '\n')
            for nn in range(len(facts)):
                if facts['food_product'][nn] == n:
                    print(facts['nutrition'][nn], ' - ',facts['pr_daily_norm'][nn], '% of Daily Value')
        similarrecipe = recipes.SimilarRecipes(list_of_ingr)
        indexes,n = similarrecipe.find_all()
        text_with_recipes = similarrecipe.top_similar()
        print('\n' + 'III. ТОП-3 ПОХОЖИХ РЕЦЕПТА:' + '\n')
        if n < len(list_of_ingr):
            print('Не нашлось рецептов со всеми этими продуктами, но вот что нашлось с максимальным их количеством: ')
            print(f"  -   {indexes['title'][0]},   рейтинг {indexes['rating'][0]}   URL: {indexes['url'][0]}")
            print(f"  -   {indexes['title'][1]},   рейтинг {indexes['rating'][1]}   URL: {indexes['url'][1]}")
            print(f"  -   {indexes['title'][2]},   рейтинг {indexes['rating'][2]}   URL: {indexes['url'][2]}")

Ingredients List Example: apple, bean, cookie, fruit, lentil, tomato


AttributeError: 'Forecast' object has no attribute 'recipes'

saffron, gin, chile pepper - bad // 
bread, avocado, cheese, garlic - great //
a, b, c, garlic - a,b,c - нет в базе//
bread, avocado, cheese, garlic, saffron, gin, chile pepper, banana, yogurt, zucchini - рецептов не найдено